 # Luonnollisen kielen käsittelyn (NLP) demo

## Lähteitä

Koodia, esimerkkejä:

- Demo mukailee esimerkkitutorialia: https://www.tensorflow.org/hub/tutorials/tf2_text_classification
- Osia myös tutorialista: https://medium.com/intro-to-artificial-intelligence/entity-extraction-using-deep-learning-8014acac6bb8

Dataa: 

- Sentimenttidata: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb
- Sentimenttidata, alkuperäinen lähde? : https://ai.stanford.edu/~amaas/data/sentiment/
- IMDb data: https://www.imdb.com/interfaces/

Menetelmiä:

- XLNet: https://github.com/zihangdai/xlnet
- XLNet-paperi: https://arxiv.org/pdf/1906.08237.pdf
- GLUE-sivusto: https://gluebenchmark.com/
- GLUE-paperi: https://openreview.net/pdf?id=rJ4km2R5t7

Muita lähteitä:

- NLP-kehitystä seuraileva GitHub-repo: https://github.com/sebastianruder/NLP-progress
- Toinen NLP-GitHub -repo: https://github.com/keon/awesome-nlp
- Turku NLP: https://turkunlp.org/

## Demo

Määritellään tarvittavat Python-kieliset paketit:

In [1]:
import os
import pickle
import numpy as np

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Kytketään pois GPU:n puuttumisesta kertovat virheviestit.

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow import keras

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.6.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


In [2]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-train.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling imdb_reviews-test.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-unsupervised.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /home/ml_user/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In [ ]:
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))

In [ ]:
train_examples[0]

In [ ]:
train_labels[0]

In [ ]:
train_examples[5]

In [ ]:
train_labels[5]

In [ ]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
# hub_layer(train_examples[:3])

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [ ]:
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

Mallin opetusta (kesto olisi n. 12 min)

In [ ]:
model_dir = '../work/nlp_model'

if os.path.exists(model_dir):
    model = keras.models.load_model(model_dir)

    with open(os.path.join(model_dir, 'hist.pkl'), 'rb') as handle:
        hist = pickle.load(handle)
else:
    history = model.fit(partial_x_train,
                        partial_y_train,
                        epochs=40,
                        batch_size=512,
                        validation_data=(x_val, y_val),
                        verbose=1)
    model.save(model_dir)
                                
    with open(os.path.join(model_dir, 'hist.pkl'), 'wb') as handle:
        pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    hist = history.history

In [ ]:
results = model.evaluate(test_data, test_labels)

In [ ]:
history_dict = hist
history_dict.keys()

In [ ]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()